In [1]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ipynb
from scipy.spatial import distance_matrix
import random
import sys as sys

In [2]:
from ipynb.fs.full.datamanagement import user_filter
from ipynb.fs.full.evaluation_workflow import split_history
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance_matrix
pd.options.mode.chained_assignment = None  # default='warn'

## Data loading

In [3]:
# Read the data
CUR_DIR = os.path.abspath('')

DATA_DIR = Path(CUR_DIR).parent / "data"
FILE_PATH = DATA_DIR/'triplets_metadata_spotify.csv'
df = pd.read_csv(FILE_PATH, index_col = 0)

In [4]:
df.head()

,user,listening_count,song_name,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year,shs_perf,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Stronger,Graduation,Kanye West,311.84934,0.877214,1.082503,2007,-1,...,-7.933,0,0.176,0.00763,0.0,0.526,0.495,103.977,4.0,2011-05-24
1,9fba771d9731561eba47216f6fbfc0023d88641b,19,Stronger,Graduation,Kanye West,311.84934,0.877214,1.082503,2007,-1,...,-7.933,0,0.176,0.00763,0.0,0.526,0.495,103.977,4.0,2011-05-24
2,85952991b8e3ca5803a08b0b2f9c6d71abf9bb5b,1,Stronger,Graduation,Kanye West,311.84934,0.877214,1.082503,2007,-1,...,-7.933,0,0.176,0.00763,0.0,0.526,0.495,103.977,4.0,2011-05-24
3,537340ff896dea11328910013cfe759413e1eeb3,2,Stronger,Graduation,Kanye West,311.84934,0.877214,1.082503,2007,-1,...,-7.933,0,0.176,0.00763,0.0,0.526,0.495,103.977,4.0,2011-05-24
4,8fce200f3912e9608e3b1463cdb9c3529aab5c08,2,Stronger,Graduation,Kanye West,311.84934,0.877214,1.082503,2007,-1,...,-7.933,0,0.176,0.00763,0.0,0.526,0.495,103.977,4.0,2011-05-24


In [5]:
# Select a working database
user_listenings = df.groupby(['user', 'song_name', 'artist_name']).agg({'listening_count' : sum}).reset_index()
users_summary = user_listenings.groupby('user').agg({'listening_count' : sum, 'song_name' : len}).reset_index()
users_summary.columns = ['user', 'listening_count', 'track_count']


## Data management

### Users filtering

In [6]:
ids = user_filter(users_summary, nlist_min = 1, nlist_max = 500, ntracks_min = 5)
wdf = df.loc[df['user'].isin(ids),:]

In [7]:
print('Nb of ligns in df  :', len(df))
print('Nb of ligns in wdf :', len(wdf))

Nb of ligns in df  : 4982520
Nb of ligns in wdf : 3863861


In [8]:
wdf['track_id'] = wdf.song_name + '-' + wdf.artist_name

### Songs management

In [8]:
# Get the songs quantitative features

tracks_feats = df.drop(['user', 'listening_count'],axis = 1)
tracks_feats['track_id'] = df.song_name + '-' + df.artist_name
tracks_quanti_feats = tracks_feats.drop(['song_name', 'release', 'artist_name',
        'artist_familiarity', 'artist_hotttnesss', 'year', 'key',
       'shs_perf', 'shs_work', 'explicit', 'mode', 'time_signature', 'release_date', 'duration', 'tempo'], axis = 1).drop_duplicates()

In [11]:
# Normalize the songs quantitative feats
X = tracks_quanti_feats.drop(['track_id'], axis = 1)
X.index = tracks_quanti_feats.track_id

scaler = StandardScaler()
X[X.columns] = pd.DataFrame(scaler.fit_transform(X), index=X.index)
#X = scaler.transform(tracks_quanti_feats)
## preserve X as a dataframe


In [10]:
# Compute the pairwise distance matrix between the songs
D = pd.DataFrame(distance_matrix(X, X), index = X.index, columns = X.index)

## Raw K-Nearest Neighboor

### Split the data into hidden and apparent sets

In [9]:
test1, test2 = split_history(wdf, 0.2, sort = True, random_state = 1234)

Sorting the triplets by users id ... Done
Computing the number of tracks listened by the users ... Done
Defining hidden tracks ids ... Done
Defining apparent tracks ids ... Done

In [ ]:
def random_k_reco(users_historic):